<a href="https://colab.research.google.com/github/ali18997/ML/blob/master/MNIST_Torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [12]:
import torch, torchvision
from torch import nn,optim
from torch.autograd import Variable as var

In [7]:
n_batch = 64
learning_rate = 0.01
n_epoch = 3
n_print = 10

In [8]:
#### Part I : Loading Your Data

T = torchvision.transforms.ToTensor()
train_data = torchvision.datasets.MNIST('mnist_data',train=True,download=True,transform=T)
val_data = torchvision.datasets.MNIST('mnist_data',train=False,download=True,transform=T)

train_dl = torch.utils.data.DataLoader(train_data,batch_size = n_batch)

val_dl = torch.utils.data.DataLoader(val_data,batch_size = n_batch)



Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw


Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
#### Part II : Writing the Network

class myCNN(nn.Module):
  def __init__(self):
    super(myCNN,self).__init__()
    self.cnn1 = nn.Conv2d(1,3,3)
    self.cnn2 = nn.Conv2d(3,2,5)
    self.linear = nn.Linear(968,10)
    self.relu = nn.ReLU()

  def forward(self,x):
    n = x.size(0)
    x = self.relu(self.cnn1(x))
    x = self.relu(self.cnn2(x))
    x = x.view(n,-1)
    x = self.linear(x)
    return x


In [13]:
#### Part III: Writing the main Training loop

mycnn = myCNN().cuda()
cec = nn.CrossEntropyLoss()
optimizer = optim.Adam(mycnn.parameters(),lr=learning_rate)

def validate(model,data):
  # To get validation accuracy = (correct/total)*100
  total = 0
  correct = 0
  for i,(images,labels) in enumerate(data):
    images = var(images.cuda())
    labels = var(labels.cuda())
    x = model(images)
    value,pred = torch.max(x,1)
    total += x.size(0)
    correct += torch.sum(pred == labels)
  return correct*100//total

for e in range(n_epoch):
  for i, (images,labels) in enumerate(train_dl): 
    images = var(images.cuda())
    labels = var(labels.cuda())
    optimizer.zero_grad()
    pred = mycnn(images)
    loss = cec(pred,labels)
    loss.backward()
    optimizer.step()
    if (i+1) % n_print == 0:
      accuracy = float(validate(mycnn, val_dl))
      print('Epoch :',e+1,'Batch :',i+1,'Loss :',float(loss.data), 'Accuracy :',accuracy,'%')


Epoch : 1 Batch : 10 Loss : 1.2082700729370117 Accuracy : 70.0 %
Epoch : 1 Batch : 20 Loss : 0.911437451839447 Accuracy : 81.0 %
Epoch : 1 Batch : 30 Loss : 0.24790768325328827 Accuracy : 86.0 %
Epoch : 1 Batch : 40 Loss : 0.1928154081106186 Accuracy : 88.0 %
Epoch : 1 Batch : 50 Loss : 0.17423272132873535 Accuracy : 87.0 %
Epoch : 1 Batch : 60 Loss : 0.29873961210250854 Accuracy : 89.0 %
Epoch : 1 Batch : 70 Loss : 0.23247960209846497 Accuracy : 90.0 %
Epoch : 1 Batch : 80 Loss : 0.2566509246826172 Accuracy : 91.0 %
Epoch : 1 Batch : 90 Loss : 0.22416161000728607 Accuracy : 91.0 %
Epoch : 1 Batch : 100 Loss : 0.1346828043460846 Accuracy : 91.0 %
Epoch : 1 Batch : 110 Loss : 0.3476770520210266 Accuracy : 92.0 %
Epoch : 1 Batch : 120 Loss : 0.2933677136898041 Accuracy : 93.0 %
Epoch : 1 Batch : 130 Loss : 0.3143053650856018 Accuracy : 93.0 %
Epoch : 1 Batch : 140 Loss : 0.34709498286247253 Accuracy : 93.0 %
Epoch : 1 Batch : 150 Loss : 0.2854737341403961 Accuracy : 92.0 %
Epoch : 1 Batc

KeyboardInterrupt: ignored

In [5]:
!nvidia-smi

Mon Sep  7 06:50:39 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.66       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     7W /  75W |      0MiB /  7611MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------